<a href="https://colab.research.google.com/github/kushagra-atre/Metal-Price-API/blob/main/API_Metal_Prices.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [18]:
from ipywidgets import Password, HBox, Label
from IPython.display import display

password_box = Password(description='API Key:')
display(HBox([Label('Enter your API key:'), password_box]))



In [19]:
import requests
import pandas as pd
from datetime import datetime, timedelta

api_key = password_box.value

start_date = datetime(2013, 1, 1)
end_date = datetime(2023, 11, 3)

data_chunks = []

while start_date < end_date:
    next_year_date = min(start_date + timedelta(days=365), end_date)

    start_date_str = start_date.strftime('%Y-%m-%d')
    next_year_date_str = next_year_date.strftime('%Y-%m-%d')


    url = f'https://api.metalpriceapi.com/v1/timeframe?api_key={api_key}&start_date={start_date_str}&end_date={next_year_date_str}&base=USD&currencies=XAG'

    response = requests.get(url)
    if response.status_code == 200:
      data = response.json()
      if data.get('success'):
        df = pd.DataFrame(data['rates']).T
        data_chunks.append(df)
      else:
            print(f"API request failed for {start_date_str} to {next_year_date_str}:", data.get('error'))
    else:
        print(f"Failed to fetch data for {start_date_str} to {next_year_date_str}:", response.status_code)

    start_date = next_year_date + timedelta(days=1)

all_data = pd.concat(data_chunks)

all_data.reset_index(inplace=True)
all_data.rename(columns={'index': 'Date', 'XAG': 'Silver Price'}, inplace=True)


all_data.to_csv('silver_prices_2013_2023.csv', index=False)

print("Data fetching complete. The data is saved to 'silver_prices_2013_2023.csv'.")



Data fetching complete. The data is saved to 'silver_prices_2013_2023.csv'.
